In [1]:
import sys, numpy as np
from tensorflow.keras.datasets import mnist

In [15]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])
one_hot_labels = np.zeros((len(labels),10))

for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
    
labels = one_hot_labels
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1

In [16]:
np.random.seed(1)

relu = lambda x: (x >= 0) * x
relu2deriv = lambda x: x>=0

lr, iterations, hidden_size, pixels_per_image, num_labels = 0.005, 350, 40, 784, 10

weights_0_1 = 0.2 * np.random.random((pixels_per_image, hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

In [10]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 += lr * layer_1.T.dot(layer_2_delta)
        weights_0_1 += lr * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write("\r"+ " I:"+str(j)+ " Error:" + str(error/float(len(images)))[0:5] +" Correct:" + str(correct_cnt/float(len(images))))
    

 I:349 Error:0.104 Correct:0.998

In [11]:
for i in range(len(test_images)):
    
    layer_0 = test_images[i:i+1]
    layer_1 = relu(np.dot(layer_0, weights_0_1))
    layer_2 = np.dot(layer_1, weights_1_2)
    
    error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
    
sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] + " Test-Acc:" + str(correct_cnt/float(len(test_images))))
print()

 Test-Err:0.742 Test-Acc:0.7873


#### There is overfitting, as the difference between train acc and test acc is over 20%

In [17]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 += lr * layer_1.T.dot(layer_2_delta)
        weights_0_1 += lr * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write("\r"+ " I:"+str(j)+ " Error:" + str(error/float(len(images)))[0:5] +" Correct:" + str(correct_cnt/float(len(images))))
    
    if(j % 10 == 0 or j == iterations-1):
        error, correct_cnt = (0.0, 0)

        for i in range(len(test_images)):

            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0, weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

            error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

        sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] + " Test-Acc:" + str(correct_cnt/float(len(test_images))))
        print()

 I:0 Error:0.722 Correct:0.537 Test-Err:0.601 Test-Acc:0.6488
 I:10 Error:0.312 Correct:0.901 Test-Err:0.420 Test-Acc:0.8114
 I:20 Error:0.260 Correct:0.937 Test-Err:0.414 Test-Acc:0.8111
 I:30 Error:0.232 Correct:0.946 Test-Err:0.417 Test-Acc:0.8066
 I:40 Error:0.215 Correct:0.956 Test-Err:0.426 Test-Acc:0.8019
 I:50 Error:0.204 Correct:0.966 Test-Err:0.437 Test-Acc:0.7982
 I:60 Error:0.194 Correct:0.967 Test-Err:0.448 Test-Acc:0.7921
 I:70 Error:0.186 Correct:0.975 Test-Err:0.458 Test-Acc:0.7864
 I:80 Error:0.179 Correct:0.979 Test-Err:0.466 Test-Acc:0.7817
 I:90 Error:0.172 Correct:0.981 Test-Err:0.474 Test-Acc:0.7758
 I:100 Error:0.166 Correct:0.984 Test-Err:0.482 Test-Acc:0.7706
 I:110 Error:0.161 Correct:0.984 Test-Err:0.489 Test-Acc:0.7686
 I:120 Error:0.157 Correct:0.986 Test-Err:0.496 Test-Acc:0.766
 I:130 Error:0.153 Correct:0.999 Test-Err:0.502 Test-Acc:0.7622
 I:140 Error:0.149 Correct:0.991 Test-Err:0.508 Test-Acc:0.758
 I:150 Error:0.145 Correct:0.991 Test-Err:0.513 Test-

#### The best test accuracy is not for the best train accuracy!!! ^

#### Dropout

In [19]:
np.random.seed(1)

relu = lambda x: (x >= 0) * x
relu2deriv = lambda x: x>=0

lr, iterations, hidden_size, pixels_per_image, num_labels = 0.005, 350, 40, 784, 10

weights_0_1 = 0.2 * np.random.random((pixels_per_image, hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1


for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 += lr * layer_1.T.dot(layer_2_delta)
        weights_0_1 += lr * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write("\r"+ " I:"+str(j)+ " Error:" + str(error/float(len(images)))[0:5] +" Correct:" + str(correct_cnt/float(len(images))))
    
    
    if(j % 10 == 0 or j == iterations-1):
        test_error, test_correct_cnt = (0.0, 0)

        for i in range(len(test_images)):

            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0, weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

        sys.stdout.write("\n" +  "I:" + str(j) + " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] + \
                         " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+" Train-Err:" + str(error/ float(len(images)))[0:5] +\
                        " Train-Acc:" + str(correct_cnt/ float(len(images))))
        print()

 I:0 Error:0.768 Correct:0.453
I:0 Test-Err:0.638 Test-Acc:0.6088 Train-Err:0.768 Train-Acc:0.453
 I:10 Error:0.341 Correct:0.877
I:10 Test-Err:0.428 Test-Acc:0.7972 Train-Err:0.341 Train-Acc:0.877
 I:20 Error:0.292 Correct:0.899
I:20 Test-Err:0.412 Test-Acc:0.8062 Train-Err:0.292 Train-Acc:0.899
 I:30 Error:0.286 Correct:0.907
I:30 Test-Err:0.415 Test-Acc:0.8079 Train-Err:0.286 Train-Acc:0.907
 I:40 Error:0.266 Correct:0.921
I:40 Test-Err:0.421 Test-Acc:0.8118 Train-Err:0.266 Train-Acc:0.921
 I:50 Error:0.274 Correct:0.925
I:50 Test-Err:0.440 Test-Acc:0.8038 Train-Err:0.274 Train-Acc:0.925
 I:60 Error:0.238 Correct:0.928
I:60 Test-Err:0.414 Test-Acc:0.8084 Train-Err:0.238 Train-Acc:0.928
 I:70 Error:0.224 Correct:0.942
I:70 Test-Err:0.401 Test-Acc:0.8109 Train-Err:0.224 Train-Acc:0.942
 I:80 Error:0.209 Correct:0.951
I:80 Test-Err:0.409 Test-Acc:0.8076 Train-Err:0.209 Train-Acc:0.951
 I:90 Error:0.202 Correct:0.958
I:90 Test-Err:0.412 Test-Acc:0.803 Train-Err:0.202 Train-Acc:0.958
 I:

#### Mini batch gradient descent

In [20]:
import numpy as np
np.random.seed(1)

def relu(x):
    return (x >= 0) * x

def relu2deriv(output):
    return output >= 0

batch_size = 64
alpha, iterations = (0.001, 300)

pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2 * np.random.random((pixels_per_image, hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

for j in range(iterations):
    
    error, correct_cnt = (0.0, 0)
    
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))
        
        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start + k : batch_start + k + 1]))
            
            layer_2_delta = (labels[batch_start:batch_end] - layer_2) / batch_size
            layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
            layer_1_delta *= dropout_mask
            
            weights_1_2 += lr * layer_1.T.dot(layer_2_delta)
            weights_0_1 += lr * layer_0.T.dot(layer_1_delta)
            
        
    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)

C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in multiply
